In [1]:
import pandas as pd
import numpy as np


Data Validation and Cleaning

Data

* Loyalty Card Number: Identification method, likely used for merging data

* Lifestate: Can be used to identify spending patterns

* Premium Customer: Way to identify wealth, could be combined wiht lifestage (renamed to make clearer)

In [2]:
pd.set_option('display.max_columns', None)
customer_info = pd.read_csv("Dataset/QVI_purchase_behaviour.csv")
customer_info

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


In [3]:
customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [4]:
customer_info = customer_info.rename(columns={"PREMIUM_CUSTOMER" : "CUSTOMER_STATUS"})
customer_info

,LYLTY_CARD_NBR,LIFESTAGE,CUSTOMER_STATUS
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


In [5]:
print(f' Customer Status: {customer_info['CUSTOMER_STATUS'].unique()}')
print(f' Lifestages: {customer_info['LIFESTAGE'].unique()}')

 Customer Status: ['Premium' 'Mainstream' 'Budget']
 Lifestages: ['YOUNG SINGLES/COUPLES' 'YOUNG FAMILIES' 'OLDER SINGLES/COUPLES'
 'MIDAGE SINGLES/COUPLES' 'NEW FAMILIES' 'OLDER FAMILIES' 'RETIREES']


Data

* Date: Needs to change type, but can be used for sales over time or which items are popular during certain times
* Store_Number: Identification for store, important when evaluating store profit
* Loyalty Card Number: Used for merging
* TXN_ID: Transaction ID
* Product Number/Product Name: For item identification
* Product quantity: Quantity of sale
* Tot Sales: Total Profit

Missing Data
* Product_Price: Calculate average price of product
* Product_Weight: Extract value from product_name

In [6]:
transaction_df = pd.read_excel("Dataset/QVI_transaction_data.xlsx")
transaction_df

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
264831,43533,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,43325,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,43410,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,43461,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [7]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


In [8]:
transaction_df_cleaned = transaction_df.copy()
transaction_df_cleaned['PROD_PRICE_$'] = transaction_df_cleaned['TOT_SALES']/transaction_df_cleaned['PROD_QTY']
transaction_df_cleaned['DATE'] = pd.to_datetime(transaction_df_cleaned['DATE'], origin='1899-12-30', unit='D') # The original value are days since the excel value
transaction_df_cleaned['YEAR'] = pd.to_datetime(transaction_df_cleaned['DATE']).dt.year
transaction_df_cleaned['MONTH'] = pd.to_datetime(transaction_df_cleaned['DATE']).dt.month
transaction_df_cleaned['PROD_WEIGHT_G'] = transaction_df_cleaned['PROD_NAME'].str.extract(r'(?i)(\d{2,4})g')[0] # The (?i) allows us to ignore capitalization, the rest allows us to extract a number 2 to 4 digits followed by g
transaction_df_cleaned['PROD_WEIGHT_G'] = transaction_df_cleaned['PROD_WEIGHT_G'].str[0:3]
transaction_df_cleaned['PROD_WEIGHT_G'] = transaction_df_cleaned['PROD_WEIGHT_G'].astype(int)
transaction_df_cleaned.sample(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PROD_PRICE_$,YEAR,MONTH,PROD_WEIGHT_G
130052,2018-11-20,184,184231,187778,102,Kettle Mozzarella Basil & Pesto 175g,2,10.8,5.4,2018,11,175
240669,2018-11-18,28,28131,25298,113,Twisties Chicken270g,2,9.2,4.6,2018,11,270
243174,2018-12-09,60,60173,56880,26,Pringles Sweet&Spcy BBQ 134g,2,7.4,3.7,2018,12,134
163432,2018-10-20,164,164154,164737,69,Smiths Chip Thinly S/Cream&Onion 175g,2,6.0,3.0,2018,10,175
98468,2018-12-31,261,261272,260851,112,Tyrrells Crisps Ched & Chives 165g,2,8.4,4.2,2018,12,165
88177,2018-07-19,105,105028,105405,114,Kettle Sensations Siracha Lime 150g,2,9.2,4.6,2018,7,150
156498,2018-07-16,72,72228,71615,93,Doritos Corn Chip Southern Chicken 150g,2,7.8,3.9,2018,7,150
230322,2019-06-29,46,46401,42252,61,Smiths Crinkle Cut Chips Chicken 170g,2,5.8,2.9,2019,6,170
23462,2019-02-19,215,215019,214189,113,Twisties Chicken270g,2,9.2,4.6,2019,2,270
42114,2019-03-02,107,107009,108390,30,Doritos Corn Chips Cheese Supreme 170g,2,8.8,4.4,2019,3,170


In [9]:
customer_transaction_merged = customer_info.merge(transaction_df_cleaned, on="LYLTY_CARD_NBR")
customer_transaction_merged

,LYLTY_CARD_NBR,LIFESTAGE,CUSTOMER_STATUS,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PROD_PRICE_$,YEAR,MONTH,PROD_WEIGHT_G
0,1000,YOUNG SINGLES/COUPLES,Premium,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,3.0,2018,10,175
1,1002,YOUNG SINGLES/COUPLES,Mainstream,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,2.7,2018,9,150
2,1003,YOUNG FAMILIES,Budget,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,3.6,2019,3,210
3,1003,YOUNG FAMILIES,Budget,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,3.0,2019,3,175
4,1004,OLDER SINGLES/COUPLES,Mainstream,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,1.9,2018,11,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264831,2370701,YOUNG FAMILIES,Mainstream,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,3.6,2018,12,210
264832,2370751,YOUNG FAMILIES,Premium,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,4.6,2018,10,150
264833,2370961,OLDER FAMILIES,Budget,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,4.2,2018,10,165
264834,2370961,OLDER FAMILIES,Budget,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,5.1,2018,10,300


In [10]:
title_order = ["LYLTY_CARD_NBR", "LIFESTAGE", "CUSTOMER_STATUS", "DATE", "YEAR", "MONTH", "STORE_NBR", "TXN_ID", "PROD_NBR", "PROD_NAME", "PROD_QTY", "TOT_SALES", "PROD_PRICE_$", "PROD_WEIGHT_G"]
customer_transaction_merged = customer_transaction_merged[title_order]
customer_transaction_merged

,LYLTY_CARD_NBR,LIFESTAGE,CUSTOMER_STATUS,DATE,YEAR,MONTH,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PROD_PRICE_$,PROD_WEIGHT_G
0,1000,YOUNG SINGLES/COUPLES,Premium,2018-10-17,2018,10,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,3.0,175
1,1002,YOUNG SINGLES/COUPLES,Mainstream,2018-09-16,2018,9,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,2.7,150
2,1003,YOUNG FAMILIES,Budget,2019-03-07,2019,3,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,3.6,210
3,1003,YOUNG FAMILIES,Budget,2019-03-08,2019,3,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,3.0,175
4,1004,OLDER SINGLES/COUPLES,Mainstream,2018-11-02,2018,11,1,5,96,WW Original Stacked Chips 160g,1,1.9,1.9,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264831,2370701,YOUNG FAMILIES,Mainstream,2018-12-08,2018,12,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,3.6,210
264832,2370751,YOUNG FAMILIES,Premium,2018-10-01,2018,10,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,4.6,150
264833,2370961,OLDER FAMILIES,Budget,2018-10-24,2018,10,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,4.2,165
264834,2370961,OLDER FAMILIES,Budget,2018-10-27,2018,10,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,5.1,300


In [11]:
customer_transaction_merged.isnull().sum() # No missing values, can begin analysis

LYLTY_CARD_NBR     0
LIFESTAGE          0
CUSTOMER_STATUS    0
DATE               0
YEAR               0
MONTH              0
STORE_NBR          0
TXN_ID             0
PROD_NBR           0
PROD_NAME          0
PROD_QTY           0
TOT_SALES          0
PROD_PRICE_$       0
PROD_WEIGHT_G      0
dtype: int64

Topics to answer
1. Has the total sales of the store improved over the years, to evaluate the financial status of the company.
2. What is the most popular item in each month and does the sale increase during months of Holidays (Thanksgiving, Christmas, etc.)
3. Which item is the most popular based on each customer status and lifestage
4. Do premium customer tend to buy in a higher quantity or higher average price
5. Which 10 product has the most value (grams per dollar)